## DSP-634 - Master targeting lists for CENTERSTONE
* source: https://achievements.atlassian.net/browse/DSP-634
* Criterion:
  * Age 18-64
  * Must reside in any of the zip codes listed in column D on this sheet.


In [2]:
import pandas as pd

In [3]:
list_zipcode_file = pd.read_csv("/repos/amezlini_Draft/Zipcodes_CENTERSTONE_Study.tsv", sep = '\t',skiprows=1)

In [4]:
list_zipcode_str = sum([sub.split(",") for sub in list_zipcode_file['Zip Codes Within 30 miles']],[])
list_zipcode = list(map(int, list_zipcode_str))
print('N =', len(list_zipcode))
print('N =', len(pd.unique(list_zipcode)))

N = 4324
N = 3390


In [5]:
%load_ext watermark
%watermark -v -p pandas

Python implementation: CPython
Python version       : 3.7.4
IPython version      : 7.14.0

pandas: 1.2.0



### Load survey snapshot for given date

In [6]:
SELECT_DATE = '2021-02-04'

In [7]:
df_demog = pd.read_parquet(f'/domino/datasets/survey_snapshots_new/{SELECT_DATE}_health_survey.parquet')
print(df_demog.shape)
print('N =', df_demog.user_id.nunique())
print(df_demog.columns)

(1277532, 19)
N = 1273730
Index(['user_id', 'gender', 'age', 'physical_health', 'quality_of_life',
       'general_health', 'mental_health', 'have_children', 'education',
       'relationship_status', 'motivation_for_joining', 'weight',
       'health_challenge', 'zipcode', 'hear_about_achievemint',
       'height_inches', 'race', 'ts', 'bmi'],
      dtype='object')


#### Number of living in the selected zipcodes

In [8]:
(df_demog.zipcode.isin(list_zipcode)).sum()

174945

#### Number of zipcodes that are in survey 

In [9]:
df_demog.zipcode[df_demog.zipcode.isin(list_zipcode)].nunique()

2767

### Age 18 - 64 -- use `groupby` to catch multiple entries per UID

In [10]:
list_uid = list(set(df_demog.query('(age >= 18) & (age <=64) & (zipcode == @list_zipcode)')['user_id']))
print('Age 18-64 & zipcode match, N = ', len(list_uid))

Age 18-64 & zipcode match, N =  172552


In [11]:
### Create random groups of equal size

In [12]:
group = 1 + (pd.Series(range(0,len(list_uid)))).mod(8).sample(n=len(list_uid), random_state=121)
data = pd.DataFrame({'participant_id':list_uid,'group':group})

### Save results to `/mnt/artifacts/results` for later download

In [13]:
data.to_csv('/mnt/results/dsp634_01_centerstone_Master_targeting_list.csv', index=None)

In [14]:
for x in range(8):
    data[data["group"] == x+1]["participant_id"].to_csv(f'/mnt/results/dsp634_01_centerstone_Master_targeting_list_{x}.csv', index=None)

In [15]:
len(list_uid)

172552

In [16]:
len(pd.array(list_uid).unique())

172552